# Análisis de Egresos Hospitalarios

En este análisis se quiere ver la cantidad de egresos hospitalarios para diversos problemas
médicos, tanto cubiertos por GES y los no GES.

En primer lugar, se quiere ver qué diagnósticos son más importantes para el Hospital del Tórax
con respecto a la cantidad de Egresos. Esto significa que una mayor cantidad de Egresos
Hospitalarios significa una mayor importancia del Hospital. Este análisis se quiere observar
acumuladamente entre 2013 y 2019, cómo también la distribución de por años.


In [1]:
import importlib

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import funciones_auxiliares_metricas_egresos as aux
import funciones_auxiliares_ranking as aux2

sns.set_theme()
plt.rcParams['figure.figsize'] = (12, 6)

importlib.reload(aux)
importlib.reload(aux2)

<module 'funciones_auxiliares_ranking' from 'c:\\Users\\ppizarro\\Desktop\\Javier Rojas\\Javier\\trabajo\\SSMO\\EPH\\modulo_gestion\\1.analisis_problemas_de_salud\\egresos-hospitalarios\\funciones_auxiliares_ranking.py'>

# Carga de Archivos Nacionales

In [2]:
AGRUPACION_DIAGNOSTICOS = ['Capítulo', 'Sección', 'Categoría', 'Descripción', 'DIAG1']
GLOSA_TORAX = 'Instituto Nacional de Enfermedades Respiratorias y Cirugía Torácica'

In [3]:
df = aux.lectura_archivos()

In [4]:
cie = aux.obtener_diccionario_cie()
completa_con_cie = pd.merge(df, cie, how='left', left_on='DIAG1', right_on='CodigoSinPunto')

## Obtener solo diagnosticos del Torax

Esto se hace, para filtrar la base de datos Nacional y solo dejar los diagnosticos que aparecen
en el Torax. Permite reducir tiempo computacional.

In [5]:
diagnosticos_torax = aux.obtener_diagnosticos_hospital(completa_con_cie, GLOSA_TORAX)
completa_diags_torax = completa_con_cie[completa_con_cie['DIAG1'].isin(diagnosticos_torax)]
completa_diags_torax['GLOSA_ESTABLECIMIENTO_SALUD'] = completa_diags_torax['GLOSA_ESTABLECIMIENTO_SALUD'].replace('Instituto Nacional de Enfermedades Respiratorias y Cirugï¿½a Torï¿½cica',
                                                                                                                   GLOSA_TORAX)

C:\Users\ppizarro\AppData\Local\Temp\ipykernel_7512\539428088.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  completa_diags_torax['GLOSA_ESTABLECIMIENTO_SALUD'] = completa_diags_torax['GLOSA_ESTABLECIMIENTO_SALUD'].replace('Instituto Nacional de Enfermedades Respiratorias y Cirugï¿½a Torï¿½cica',


# Cálculo de métricas para todos los Hospitales Nacionales

Se calculan las siguientes métricas:

- Número de Egresos Hospitalarios (**n_egresos**)
- Dias de Estada Promedio (**dias_estada_promedio**)
- Cantidad de Intervenciones Quirúrgicas (**n_int_q**)
- Cantidad de Fallecidos (**n_muertos**)

Todo lo anterior se calcula con un nivel de desagregación de AÑO - HOSPITAL - DIAGNOSTICO

In [6]:
agrupacion_anio = (['ANO_EGRESO', 'ESTABLECIMIENTO_SALUD', 'GLOSA_ESTABLECIMIENTO_SALUD'] + 
                   AGRUPACION_DIAGNOSTICOS)

metricas_nacionales = aux.calcular_metricas_de_egresos_agrupados(completa_diags_torax, 
                                                                 agrupacion_anio)

# Cálculo de Rankings para cada uno los Hospitales según distintas variables

Se quiere ver el Ranking para cada uno de los Hospitales según las variables calculadas
previamente. Además, se quiere ver el Ranking de cada Hospital en distintos estratos. Los
estratos a analizar son:

- Nacional (Todos los Hospitales)
- Público (Los que pertenecen al SNSS)
- Privado (Los que NO perteneces al SNSS + Tórax)
- GRD (Hospitales que tengan GRD)
- Interno (Interno de cada hospital)

## Hacer el análisis

In [7]:
estratos_a_calcular = aux2.obtener_codigos_de_estratos(completa_diags_torax, 112103)
variables_a_rankear = ['n_egresos']
subgrupo_del_ranking = ['ANO_EGRESO', 'DIAG1']
unir_tablas_por = agrupacion_anio

resumen_rankings = aux2.realizar_ranking_por_estrato(metricas_nacionales, estratos_a_calcular,
                                                     variables_a_rankear,
                                                     subgrupo_del_ranking, unir_tablas_por)


In [8]:
resumen_rankings.to_csv('output/rankings_hospitales_maestro.csv', encoding='latin-1',
                        decimal=',', sep=';')
resumen_rankings.to_excel('output/rankings_hospitales_maestro.xlsx')

# Analisis Sociodemografico del Torax

In [10]:
torax_con_cie['EDAD_CATEGORIA'] = pd.cut(completa_con_cie['EDAD_A_OS'], 
                                         bins=pd.interval_range(start=0, 
                                                                end=100, freq=10, closed='left'))

NameError: name 'torax_con_cie' is not defined

In [ ]:
agrupacion_diagnostico = ['ANO_EGRESO', 'DIAG1']

diccionario_sociodemografico = {
    'SEXO': ['SEXO'],
    'EDAD': ['EDAD_CATEGORIA'],
    'REGION': ['REGION_RESIDENCIA', 'GLOSA_REGION_RESIDENCIA'],
    'COMUNA': ['COMUNA_RESIDENCIA', 'GLOSA_COMUNA_RESIDENCIA'],
    'PUEBLO_ORIGINARIO': ['PUEBLO_ORIGINARIO'],
    'PAIS_ORIGEN': ['PAIS_ORIGEN', 'GLOSA_PAIS_ORIGEN'],
    'PREVISION': ['PREVISION', 'BENEFICIARIO']
}


with pd.ExcelWriter('output/ranking_torax_pais/distribucion_sociodemografica.xlsx') as file:
    for nombre_agrupacion, vars_agregacion in diccionario_sociodemografico.items():
        resumen_agrupacion = torax_con_cie.groupby(agrupacion_diagnostico)[vars_agregacion].value_counts()
        resumen_agrupacion.name = 'conteo'
        resumen_agrupacion = resumen_agrupacion.reset_index()
        resumen_agrupacion['llave'] = resumen_agrupacion['ANO_EGRESO'].astype(str) + '-' + resumen_agrupacion['DIAG1'].astype(str)
        resumen_agrupacion.to_excel(file, sheet_name=nombre_agrupacion, index=False)
